In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten, BatchNormalization
from keras.callbacks import EarlyStopping, CSVLogger

from loader import load_20news
from custom_layer import MonteCarloLRF, SeparableMonteCarloLRF, SeparableMonteCarloMaxPoolingV2, RandomLRF

import numpy as np
import sklearn as sk
import pickle

/home/vle/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
top_words=10000
sparse=False
remove_short_documents=True
notebook = 'mcNet_top10k_temptative_42'

In [4]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=top_words, sparse=sparse,
                remove_short_documents=remove_short_documents, verbose=False)

In [5]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
num_classes = Y_train.shape[1]

In [6]:
%%time
#Process next cell only once
path = os.path.join('probabilities_' + 
                    'top' + str(top_words) +
                    '_sparse' + str(sparse) +
                    '_removeShorts' + str(remove_short_documents) +
                    '.pkl')
if os.path.isfile(path):
  probabilities = pickle.load(open(path, "rb"), encoding='latin1')

CPU times: user 2.38 s, sys: 800 ms, total: 3.18 s
Wall time: 3.17 s


In [7]:
if not(os.path.isfile(path)):
  METRIC = 'cosine'#'euclidean'
  distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

  # enforce exact zero
  for k in xrange(distances.shape[0]):
    distances[k,k] = 0.

  # max normalize
  #distances /= distances.max()
  distances /= distances.max(axis=1).reshape((distances.shape[0], 1))

  # use tricube kernel (becaause of flatness around 0)
  probabilities = (1. - np.abs(distances) ** 3) ** 3

  # remove auto connections (which are taken anyway in LRF)
  for k in xrange(probabilities.shape[0]):
    probabilities[k,k] = 0.

  # normalize proba
  probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))
  
  # pickled for later use
  pickle.dump(probabilities, open(path,"wb"))  

In [8]:
batch_size = 256
model = Sequential()
model.add(RandomLRF(probabilities, LRF_size=16, filters=25, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(BatchNormalization(axis=1))
#model.add(Conv1D(1, kernel_size=1, activation='relu',
#                 padding='same',kernel_initializer='he_uniform'))
#model.add(SeparableMonteCarloMaxPoolingV2(LRF_size=4, new_size=2500))
#model.add(Dropout(0.2))
model.add(Flatten())
#model.add(Dense(500, activation='relu'))
model.add(Dropout(0.8))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
random_lrf_1 (RandomLRF)     (None, 10000, 25)         425       
_________________________________________________________________
batch_normalization_1 (Batch (None, 10000, 25)         40000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 250000)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 250000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                5000020   
Total params: 5,040,445
Trainable params: 5,020,445
Non-trainable params: 20,000
_________________________________________________________________


In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopper = EarlyStopping(min_delta=0.001, patience=2)
csv = CSVLogger(notebook + '_log.csv')

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    callbacks=[early_stopper, csv],
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 10168 samples, validate on 7071 samples
Epoch 1/20
10168/10168 [==============================] - 11s 1ms/step - loss: 1.6425 - acc: 0.5808 - val_loss: 1.2168 - val_acc: 0.6756
Epoch 2/20
10168/10168 [==============================] - 9s 879us/step - loss: 0.4605 - acc: 0.8870 - val_loss: 1.1700 - val_acc: 0.6921
Epoch 3/20
10168/10168 [==============================] - 9s 879us/step - loss: 0.2266 - acc: 0.9469 - val_loss: 1.2157 - val_acc: 0.6907
Epoch 4/20
10168/10168 [==============================] - 9s 879us/step - loss: 0.1243 - acc: 0.9762 - val_loss: 1.3969 - val_acc: 0.6725
Test loss: 1.3969361979154529
Test accuracy: 0.6724649979208066


In [10]:
%%bash
tail mcNet_top10k_temptative_42_log.csv

epoch,acc,loss,val_acc,val_loss
0,0.5808418563836021,1.6425235706647487,0.6755762969043194,1.2167736569888654
1,0.8869984262951897,0.4604991487777674,0.6921227553864885,1.169997093844761
2,0.9468922109513842,0.22660820962264724,0.6907085280424465,1.215745747671018
3,0.9761998427373795,0.12425905712869204,0.6724649974487573,1.3969360224311516
